# Todo list: 
<li> Find a way to "trigger" messege to kafka (May not need to store in hdfs anymore-> contr)</li>
<li> Push to github </li>

In [ ]:
!wget http://central.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.11/2.4.1/spark-sql-kafka-0-10_2.11-2.4.1.jar

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.types import StructType
import json
emailSchema = StructType() \
        .add("subject", StringType()) \
        .add("body", StringType())

# Group the data by window and subject and compute the count of each group

<li>The engine will maintain state and allow late data to be update the state until (max event time seen by the engine - late threshold > 1 minute). In other words, late data within the threshold will be aggregated, but data later than the threshold will be dropped.</li> 
<li>Count subject within 5 minutes windows, updating every 5 sec</li>
<li>Grouping is 2 min delay on Spark</li>

In [2]:

# Subscribe to 1 topic

# .option("kafka.bootstrap.servers", "188.185.79.229:9092")\
# .option("startingOffsets", "earliest") \

raw_data = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "188.185.79.229:9092")\
.option("subscribe","mock_email_alert")\
.load()\
.select(
    from_json(col("value").cast("string"), emailSchema).getField("subject").alias("subject"),
    col("timestamp").alias("timestamp"))\

raw_data.printSchema()

root
 |-- subject: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



## Pattern 
<li>Similar Subject name more than N number per window in "filtered_email"</li>

<li>Different Subject name more than N number per window in "strenge_email"</li>

In [3]:
groupped_data=raw_data\
.withWatermark("timestamp", "1 minute")\
.groupBy(window('timestamp', "1 minute", "1 minute"),"subject")\
.agg(count("subject"))\

groupped_data.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- subject: string (nullable = true)
 |-- count(subject): long (nullable = false)



In [99]:
unique_row=raw_data\
.select(
    window("timestamp", "1 minute"),
    "timestamp",
    "subject"
  )\
.withWatermark("window", "1 day")\
.dropDuplicates(["subject", "window"])\

In [5]:
filtered_data= groupped_data.filter("count(subject) > 1")\

In [107]:
streamout_data=filtered_data\
.select( \
    to_json(struct("window")).alias("key"),
    col("subject").cast("string").alias("value")) \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "188.185.79.229:9092") \
  .option("topic", "filter_alert") \
  .option("checkpointLocation", "/user/ysunthor/checkpoint") \
  .outputMode("Append") \

In [101]:
raw_data_flow = raw_data.writeStream.queryName("email").outputMode("Append").format("memory").start()
# groupped_data_flow = groupped_data.writeStream.queryName("groupped_email").outputMode("Append").format("memory").start()
# filtered_data_flow = filtered_data.writeStream.queryName("filtered_email").outputMode("Append").format("memory").start()
unique_row_flow=unique_row.writeStream.queryName("unique_email").outputMode("Append").format("memory").start()

# streamout_data_flow = streamout_data.start()

In [274]:
# raw_data.isStreaming
unique_row.isStreaming
# filtered_data.isStreaming
# groupped_data.isStreaming

True

In [109]:
spark.streams.active

In [110]:
# raw_data_flow.lastProgress
# groupped_data_flow.lastProgress
unique_row_flow.lastProgress
# filtered_data_flow.lastProgress
# streamout_data_flow.lastProgress

{'id': 'f59f1d13-5347-4073-89c4-7224b1c715c4',
 'runId': '808da349-4957-4ad9-82cb-2aad1424a06d',
 'name': 'unique_email',
 'timestamp': '2019-06-13T14:56:28.119Z',
 'batchId': 0,
 'numInputRows': 0,
 'processedRowsPerSecond': 0.0,
 'durationMs': {'addBatch': 61455,
  'getBatch': 0,
  'getEndOffset': 0,
  'queryPlanning': 72,
  'setOffsetRange': 179,
  'triggerExecution': 62098,
  'walCommit': 162},
 'eventTime': {'watermark': '1970-01-01T00:00:00.000Z'},
 'stateOperators': [{'numRowsTotal': 0,
   'numRowsUpdated': 0,
   'memoryUsedBytes': 44599,
   'customMetrics': {'loadedMapCacheHitCount': 0,
    'loadedMapCacheMissCount': 0,
    'stateOnCurrentVersionSizeBytes': 15799}}],
 'sources': [{'description': 'KafkaV2[Subscribe[mock_email_alert]]',
   'startOffset': None,
   'endOffset': {'mock_email_alert': {'0': 3421}},
   'numInputRows': 0,
   'processedRowsPerSecond': 0.0}],
 'sink': {'description': 'MemorySink'}}

In [100]:
raw_data_flow.stop()
# groupped_data_flow.stop()
# filtered_data_flow.stop()
unique_row_flow.stop()
# streamout_data_flow.stop()

In [112]:
alerts = spark.sql("select * from email")
alerts.show()

+--------------+--------------------+
|       subject|           timestamp|
+--------------+--------------------+
|      CRON job|2019-06-13 16:56:...|
|       WARNING|2019-06-13 16:56:...|
|PopDB problems|2019-06-13 16:56:...|
|PopDB problems|2019-06-13 16:56:...|
|PopDB problems|2019-06-13 16:56:...|
|PopDB problems|2019-06-13 16:56:...|
|PopDB problems|2019-06-13 16:57:...|
|      CRON job|2019-06-13 16:57:...|
|      CRON job|2019-06-13 16:57:...|
|      CRON job|2019-06-13 16:57:...|
|      CRON job|2019-06-13 16:57:...|
|      CRON job|2019-06-13 16:57:...|
|       WARNING|2019-06-13 16:57:...|
|      CRON job|2019-06-13 16:57:...|
+--------------+--------------------+



In [281]:
alerts = spark.sql("select * from groupped_email")
alerts.show()

+--------------------+--------------+--------------+
|              window|       subject|count(subject)|
+--------------------+--------------+--------------+
|[2019-06-13 14:56...|      CRON job|             1|
|[2019-06-13 14:56...|PopDB problems|             2|
|[2019-06-13 14:57...|PopDB problems|             6|
|[2019-06-13 14:57...|      CRON job|             2|
|[2019-06-13 14:57...|       WARNING|             4|
|[2019-06-13 14:58...|PopDB problems|             4|
|[2019-06-13 14:58...|      CRON job|             6|
|[2019-06-13 14:58...|       WARNING|             2|
|[2019-06-13 14:59...|       WARNING|             1|
|[2019-06-13 14:59...|PopDB problems|             9|
|[2019-06-13 14:59...|      CRON job|             2|
|[2019-06-13 15:00...|       WARNING|             3|
|[2019-06-13 15:00...|PopDB problems|             3|
|[2019-06-13 15:00...|      CRON job|             6|
|[2019-06-13 15:01...|      CRON job|             2|
|[2019-06-13 15:01...|       WARNING|         

In [24]:
alerts = spark.sql("select * from filtered_email")
alerts.show()

AnalysisException: 'Table or view not found: filtered_email; line 1 pos 14'

In [115]:
alerts = spark.sql("select * from unique_email")
alerts.show()

+--------------------+--------------------+--------------+
|              window|           timestamp|       subject|
+--------------------+--------------------+--------------+
|[2019-06-13 16:56...|2019-06-13 16:56:...|PopDB problems|
|[2019-06-13 16:56...|2019-06-13 16:56:...|      CRON job|
|[2019-06-13 16:56...|2019-06-13 16:56:...|       WARNING|
|[2019-06-13 16:57...|2019-06-13 16:57:...|      CRON job|
|[2019-06-13 16:57...|2019-06-13 16:57:...|PopDB problems|
|[2019-06-13 16:57...|2019-06-13 16:57:...|       WARNING|
|[2019-06-13 16:58...|2019-06-13 16:58:...|PopDB problems|
|[2019-06-13 16:58...|2019-06-13 16:58:...|      CRON job|
+--------------------+--------------------+--------------+

